# Data Cleaning for Cafe Sales 

# Set up and load the data

In [104]:
# Import the libraries and load the data
import pandas as pd
import numpy as np
df = pd.read_csv("dirty_cafe_sales.csv")


In [105]:
# let's look at the data
df.head(5)
df.shape

(10000, 8)

In [106]:
#Structure & data quality check

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [107]:
# Convert column names to snake-case 
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(' ', '_')
      .str.replace('-', '_'))
df.columns

Index(['transaction_id', 'item', 'quantity', 'price_per_unit', 'total_spent',
       'payment_method', 'location', 'transaction_date'],
      dtype='object')

# Handling the date column

**The transaction date column contained invalid string values. These were handled using pd.to_datetime with error coercion, converting malformed entries to nulls to ensure reliable time-based analysis.**

In [108]:
df['transaction_date'].isna().sum()

df[df['transaction_date'].isna()]

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
77,TXN_2091733,Salad,1,5,5,NaN,In-store,NaN
104,TXN_7447872,Juice,2,NaN,6,NaN,NaN,NaN
160,TXN_1093800,Sandwich,3,4,12,Cash,Takeaway,NaN
175,TXN_6463132,Cookie,5,1,5,Credit Card,Takeaway,NaN
246,TXN_1908636,Tea,2,1.5,3,NaN,UNKNOWN,NaN
...,...,...,...,...,...,...,...,...
9769,TXN_9686177,Cake,3,3,9,NaN,In-store,NaN
9833,TXN_5536245,Smoothie,4,4,16,Cash,NaN,NaN
9885,TXN_4659954,NaN,3,4,12,Credit Card,In-store,NaN
9931,TXN_8344810,Smoothie,2,4,8,NaN,UNKNOWN,NaN


In [109]:
#Invalid date values were identified and safely handled using coercion.
df['transaction_date'] = pd.to_datetime(
    df['transaction_date'],
    format='%m/%d/%Y',
    errors='coerce'
)

# Checking nulls and invalid values in numeric columns

**The quantity, price_per_unit and total_spent fields contained invalid string entries. These were handled using numeric coercion and converted to nullable integers/float to preserve data integrity.**

In [110]:
df[df['quantity'].isna()]

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
66,TXN_8501819,Juice,NaN,3,6,Cash,NaN,2023-03-30
341,TXN_2265316,Cookie,NaN,1,3,Credit Card,In-store,2023-12-29
376,TXN_6319728,Coffee,NaN,2,4,Credit Card,In-store,2023-07-18
412,TXN_4660753,Juice,NaN,3,3,Credit Card,Takeaway,2023-10-04
532,TXN_7533411,Cookie,NaN,1,1,Digital Wallet,In-store,2023-11-09
...,...,...,...,...,...,...,...,...
9634,TXN_8436045,Cake,NaN,3,15,Credit Card,NaN,2023-08-08
9844,TXN_4528914,Salad,NaN,5,5,ERROR,In-store,2023-08-06
9869,TXN_1975184,Coffee,NaN,2,UNKNOWN,Digital Wallet,NaN,2023-01-15
9887,TXN_8963470,Salad,NaN,5,10,NaN,In-store,2023-06-01


In [111]:
df['quantity'] = (
    pd.to_numeric(df['quantity'], errors='coerce')
      .astype('Int64')
)

In [112]:
df['price_per_unit'] = (
    pd.to_numeric(df['price_per_unit'], errors='coerce')
      .astype('Float64')
)

In [113]:
df['total_spent'] = (
    pd.to_numeric(df['total_spent'], errors='coerce')
      .astype('Float64')
)

In [114]:
#Find rows where any numeric field is invalid
df[(df['price_per_unit'] <= 0) | (df['quantity'] <= 0) |(df['total_spent'] <= 0)]

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date


# Cleaning string type columns 

In [115]:
#Strip the string type column
df['transaction_id'] = df['transaction_id'].str.strip().astype('string')

#Strip and check unique values
str_cols = ['item', 'payment_method', 'location']
for col in str_cols:
    df[col] = df[col].str.strip().str.lower().astype('string')
    print(f"{col} : {df[col].unique()} \n")

item : <StringArray>
[  'coffee',     'cake',   'cookie',    'salad', 'smoothie',  'unknown',
 'sandwich',       <NA>,    'error',    'juice',      'tea']
Length: 11, dtype: string 

payment_method : <StringArray>
['credit card', 'cash', 'unknown', 'digital wallet', 'error', <NA>]
Length: 6, dtype: string 

location : <StringArray>
['takeaway', 'in-store', 'unknown', <NA>, 'error']
Length: 5, dtype: string 



In [116]:
#Data Standardization
for col in str_cols:
    df[col] = df[col].replace('error', 'unknown').fillna('unknown')
    print(f"{col} : {sorted(df[col].unique())} \n")

item : ['cake', 'coffee', 'cookie', 'juice', 'salad', 'sandwich', 'smoothie', 'tea', 'unknown'] 

payment_method : ['cash', 'credit card', 'digital wallet', 'unknown'] 

location : ['in-store', 'takeaway', 'unknown'] 



**Categorical fields such as payment method and location were retained and labeled as ‘Unknown’ to preserve transaction volume and highlight data quality limitations.**

# Checking Duplicates

In [117]:
df.duplicated().sum()

np.int64(0)

# Imputation for quantity, price per unit and total spent fields

In [118]:
#Imputated quantiy where price_per_unit and total_spent are available
mask_1 = (
    df['quantity'].isna() &
    df['total_spent'].notna() &
    df['price_per_unit'].notna() &
    (df['price_per_unit'] != 0)
)

df.loc[mask_1, 'quantity'] = df.loc[mask_1, 'total_spent'] / df.loc[mask_1, 'price_per_unit']


In [119]:
#Imputated price_per_unit where quantiy and total_spent are available
mask_2 = (
    df['price_per_unit'].isna() &
    df['total_spent'].notna() &
    df['quantity'].notna() &
    (df['quantity'] != 0)
)

df.loc[mask_2, 'price_per_unit'] = df.loc[mask_2, 'total_spent'] / df.loc[mask_2, 'quantity']

In [120]:
#Imputated total_spent where quantiy and price_per_unit are available
mask_3 = (
    df['total_spent'].isna() &
    df['price_per_unit'].notna() &
    df['quantity'].notna() &
    (df['quantity'] != 0)
)

df.loc[mask_3, 'total_spent'] = df.loc[mask_3, 'price_per_unit'] * df.loc[mask_3, 'quantity']

In [121]:
df = df.dropna(subset=['quantity', 'price_per_unit', 'total_spent'])

**Rows with missing numerical values (unable to be imputated) affecting revenue calculations were removed.**


In [122]:
df.isna().sum()

transaction_id        0
item                  0
quantity              0
price_per_unit        0
total_spent           0
payment_method        0
location              0
transaction_date    457
dtype: int64

In [123]:
#date range
df['transaction_date'].describe()

count                             9485
mean     2023-07-01 23:58:10.690564096
min                2023-01-01 00:00:00
25%                2023-04-01 00:00:00
50%                2023-07-02 00:00:00
75%                2023-10-02 00:00:00
max                2023-12-31 00:00:00
Name: transaction_date, dtype: object

In [124]:
#Statistical summary
df.info()
df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
Index: 9942 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    9942 non-null   string        
 1   item              9942 non-null   string        
 2   quantity          9942 non-null   Int64         
 3   price_per_unit    9942 non-null   Float64       
 4   total_spent       9942 non-null   Float64       
 5   payment_method    9942 non-null   string        
 6   location          9942 non-null   string        
 7   transaction_date  9485 non-null   datetime64[ns]
dtypes: Float64(2), Int64(1), datetime64[ns](1), string(4)
memory usage: 728.2 KB


,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
count,9942,9942,9942.0,9942.0,9942.0,9942,9942,9485
unique,9942,9,<NA>,<NA>,<NA>,4,3,NaN
top,TXN_4433211,juice,<NA>,<NA>,<NA>,unknown,unknown,NaN
freq,1,1167,<NA>,<NA>,<NA>,3158,3940,NaN
mean,NaN,NaN,3.025749,2.947848,8.931855,NaN,NaN,2023-07-01 23:58:10.690564096
min,NaN,NaN,1.0,1.0,1.0,NaN,NaN,2023-01-01 00:00:00
25%,NaN,NaN,2.0,2.0,4.0,NaN,NaN,2023-04-01 00:00:00
50%,NaN,NaN,3.0,3.0,8.0,NaN,NaN,2023-07-02 00:00:00
75%,NaN,NaN,4.0,4.0,12.0,NaN,NaN,2023-10-02 00:00:00
max,NaN,NaN,5.0,5.0,25.0,NaN,NaN,2023-12-31 00:00:00


In [125]:
#Export cleaned data
#df.to_csv("cafe_sales_cleaned_dataset.csv",index=False)    